In [2]:
client = Client("tcp://169.228.130.5:17763")
client

Client Scheduler: tcp://169.228.130.5:17763 Dashboard: http://169.228.130.5:13346/status,Cluster Workers: 102 Cores: 102 Memory: 408.00 GB


## cross section

In [1]:
xs = {
    "DYJets": 5941.0,
    "ttbar": 831.76, 
    "ZG": 55.6, 
    "WG": 191.4,
    "GJets_HT40To100": 23100.0, 
    "GJets_HT100To200": 8631.0,
    "GJets_HT200To400": 2280.0,
    "GJets_HT400To600": 273.0,
    "GJets_HT600ToInf": 94.5,
    "QCD_pT30To40": 24810.0,
    "QCD_pT40ToInf": 113400.0,
    "Diphoton": 84.4,
    "ZH": 0.002006453, #https://github.com/cms-analysis/flashgg/blob/dev_legacy_runII/MetaData/data/cross_sections.json
    "signal": 0.0098 #fb, all bkg are in pb
} 
xs

{'DYJets': 5941.0,
 'ttbar': 831.76,
 'ZG': 55.6,
 'WG': 191.4,
 'GJets_HT40To100': 23100.0,
 'GJets_HT100To200': 8631.0,
 'GJets_HT200To400': 2280.0,
 'GJets_HT400To600': 273.0,
 'GJets_HT600ToInf': 94.5,
 'QCD_pT30To40': 24810.0,
 'QCD_pT40ToInf': 113400.0,
 'Diphoton': 84.4,
 'ZH': 0.002006453,
 'signal': 0.0098}

In [8]:
import json
filename = "./metadata/xsection.json"
with open(filename, "w") as f:
    json.dump(xs, f, indent=4)

## total weight

In [4]:
def get_totWeight(args):
    fname = args[0]
    f = uproot4.open(fname)
    t = f["Runs"]
    return np.sum(t["genEventSumw"].array())

In [468]:
futures_w = {}
results_w = {}

for key in chunks:
    if "EGamma" in key: continue
    print ("start process {}".format(key))
    futures_w[key] = client.map(get_totWeight, chunks[key], retries=5)
    
    results_local = []
    while len(results_local) < len(chunks[key]): 
        ac = as_completed(futures_w[key], with_results=True)
        
        for future, result in ac:
            results_local.append(result)
            
    results_w[key] = np.sum(np.array(results_local))

start process DYJets
start process ttbar
start process ZG
start process WG
start process GJets_HT40To100
start process GJets_HT100To200
start process GJets_HT200To400
start process GJets_HT400To600
start process GJets_HT600ToInf
start process QCD_pT30To40
start process QCD_pT40ToInf
start process Diphoton
start process ZH
start process signal


In [6]:
results_w = {'DYJets': 17799598587.564648,
 'ttbar': 284532045536.7458,
 'ZG': 1514115614.2427294,
 'WG': 6107452.571461948,
 'GJets_HT40To100': 9370096.751511302,
 'GJets_HT100To200': 9795369.458845828,
 'GJets_HT200To400': 19051574.496664558,
 'GJets_HT400To600': 4650962.691182764,
 'GJets_HT600ToInf': 4970069.563275842,
 'QCD_pT30To40': 14526026.0,
 'QCD_pT40ToInf': 20622034.0,
 'Diphoton': 6423331.3,
 'ZH': 1740.8696149528298,
 'signal': 997898.0}

In [7]:
filename_w = "./metadata/totalWeights.json"
with open(filename_w, "w") as f:
    json.dump(results_w, f, indent=4)

## file chunks

In [9]:
%%time

filepath = "/hadoop/cms/store/user/legianni/ProjectMetis/" 
skim_version = "x3"

DY_files = filepath + "DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8_18____" + skim_version + "/*root"
ttbar_files = filepath + "TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8_18____" + skim_version + "/*root"
ZG_files = filepath + "ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8_18_3____" + skim_version + "/*root"
WG_files = filepath + "WGToLNuG_TuneCP5_13TeV-madgraphMLM-pythia8_18____" + skim_version + "/*root"
GJets_HT40To100_files = filepath + "GJets_HT-40To100_TuneCP5_13TeV-madgraphMLM-pythia8_18____" + skim_version + "/*root"
GJets_HT100To200_files = filepath + "GJets_HT-100To200_TuneCP5_13TeV-madgraphMLM-pythia8_18____" + skim_version + "/*root"
GJets_HT200To400_files = filepath + "GJets_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8_18____" + skim_version + "/*root"
GJets_HT400To600_files = filepath + "GJets_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8_18____" + skim_version + "/*root"
GJets_HT600ToInf_files = filepath + "GJets_HT-600ToInf_TuneCP5_13TeV-madgraphMLM-pythia8_18____" + skim_version + "/*root"
QCD_pT30To40_files = filepath + "QCD_Pt-30to40_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8_17____" + skim_version + "/*root"
QCD_pT40ToInf_files = filepath + "QCD_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8_17____" + skim_version + "//*root"
Diphoton_files = filepath + "DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa_18____" + skim_version + "/*root"

import glob
background_fileset = {
    "DYJets": glob.glob(DY_files),
    "ttbar": glob.glob(ttbar_files),
    "ZG": glob.glob(ZG_files),
    "WG": glob.glob(WG_files),
    "GJets_HT40To100": glob.glob(GJets_HT40To100_files),
    "GJets_HT100To200": glob.glob(GJets_HT100To200_files),
    "GJets_HT200To400": glob.glob(GJets_HT200To400_files),
    "GJets_HT400To600": glob.glob(GJets_HT400To600_files),
    "GJets_HT600ToInf": glob.glob(GJets_HT600ToInf_files),
    "QCD_pT30To40": glob.glob(QCD_pT30To40_files),
    "QCD_pT40ToInf": glob.glob(QCD_pT40ToInf_files),
    "Diphoton": glob.glob(Diphoton_files)
}

background_res_fileset = {
    "ZH":glob.glob(filepath + "ggZH_HToGG_ZToLL_M125_TuneCP5_13TeV-powheg-pythia8_18____" + skim_version + "/*.root"),
} 

signal_fileset = {
    "signal":glob.glob(filepath + "HHggtautau_Era2017____x3/*.root"),
}

data_fileset = {
    "EGamma_2018A":glob.glob(filepath + "EGamma_Run2018A____" + skim_version + "/*root"),
    "EGamma_2018B":glob.glob(filepath + "EGamma_Run2018B____" + skim_version + "/*root"),
    "EGamma_2018C":glob.glob(filepath + "EGamma_Run2018C____" + skim_version + "/*root"),
    "EGamma_2018D":glob.glob(filepath + "EGamma_Run2018D____" + skim_version + "/*root")
}

filesets = {"data": data_fileset,
            "background": background_fileset,
            "background_res": background_res_fileset,
            "signal": signal_fileset}

CPU times: user 10.3 ms, sys: 2.45 ms, total: 12.8 ms
Wall time: 454 ms


In [11]:
%%time
import utils

chunk_size = 5e5
chunks = {}
tot_evts = 0
tot_chunks = 0

for cat in ["data", "background", "background_res", "signal"]:
    fileset = filesets[cat]
    for key in fileset:
        fnames = fileset[key]
        cks, evts = utils.get_chunking(tuple(fnames),chunk_size,treename="Events")
        chunks[key] = cks
        tot_evts += evts
        tot_chunks += len(cks)
        print("{}: total {} events, got {} files, return {} chunks".format(key, evts, len(fnames), len(cks))) 
print()
#print(chunks.keys())
print ("total events to be processed: {0:8.2e}".format(tot_evts) )
print ("total chunks: {}".format(tot_chunks))

/home/users/hmei/miniconda3/envs/daskworkerenv2/lib/python3.7/site-packages/uproot3/__init__.py:138: FutureWarning: Consider switching from 'uproot3' to 'uproot', since the new interface became the default in 2020.

    pip install -U uproot

In Python:

    >>> import uproot
    >>> with uproot.open(...) as file:
    ...

  FutureWarning
/home/users/hmei/miniconda3/envs/daskworkerenv2/lib/python3.7/site-packages/awkward0/__init__.py:23: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  FutureWarning


EGamma_2018A: total 93164815 events, got 239 files, return 239 chunks
EGamma_2018B: total 50705031 events, got 98 files, return 155 chunks
EGamma_2018C: total 49484413 events, got 110 files, return 118 chunks
EGamma_2018D: total 229268651 events, got 426 files, return 782 chunks
DYJets: total 158051 events, got 1 files, return 1 chunks
ttbar: total 35821609 events, got 167 files, return 167 chunks
ZG: total 4735292 events, got 14 files, return 14 chunks
WG: total 435890 events, got 6 files, return 6 chunks
GJets_HT40To100: total 580125 events, got 7 files, return 7 chunks
GJets_HT100To200: total 1422209 events, got 9 files, return 9 chunks
GJets_HT200To400: total 4925264 events, got 16 files, return 16 chunks
GJets_HT400To600: total 1384436 events, got 7 files, return 7 chunks
GJets_HT600ToInf: total 1522839 events, got 9 files, return 9 chunks
QCD_pT30To40: total 1109922 events, got 9 files, return 9 chunks
QCD_pT40ToInf: total 3152328 events, got 13 files, return 13 chunks
Diphoton: 

In [22]:
filename_chunk = "./metadata/chunklist.json"
with open(filename_chunk, "w") as f:
    json.dump(chunks, f, indent=4)